In [1]:
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
file_name = 'data_256.pkl'
image_data, sentiment_data, sentence_data, sentence_tags, hashtag_data = pickle.load(open(file_name, 'rb'))

In [3]:
size = 1500

image_data = image_data[:size]
sentiment_data = sentiment_data[:size]
sentence_data = sentence_data[:size]
sentence_tags = sentence_tags[:size]
hashtag_data = hashtag_data[:size]

In [4]:
print(image_data.shape)
print(sentiment_data.shape)
print(sentence_data.shape)
print(len(sentence_tags))

(1500, 256, 256, 3)
(1500, 4)
(1500,)
1500


In [5]:
from konlpy.tag import Komoran

komoran = Komoran()

output_tags = [[p[0] for p in komoran.pos(s)] for s in tqdm(sentence_data)]

100%|██████████| 1500/1500 [00:01<00:00, 1116.14it/s]


In [6]:
image_words = set()
for tags in sentence_tags:
    for tag in tags:
        if tag[0] not in image_words:
            image_words.add(tag[0])
            
image_words = sorted(list(image_words))
n_image_words = len(image_words)

w2i_image = {w:i for i, w in enumerate(image_words)}

image_word_data = np.zeros(shape=(len(sentence_tags), n_image_words), dtype='float32')

for i, tags in enumerate(sentence_tags):
    for tag in tags:
        image_word_data[i, w2i_image[tag[0]]] += 1

In [7]:
output_words = set()
for tags in output_tags:
    tags.insert(0, '\t')
    tags.append('\n')
    for tag in tags:
        if tag not in output_words:
            output_words.add(tag)
            
output_words = sorted(list(output_words))
output_words[0], output_words[1] = output_words[1], output_words[0]
n_output_words = len(output_words)
max_seq_length = max(map(len, output_tags))

w2i_output = {w:i for i, w in enumerate(output_words)}

input_data = np.zeros(shape=(len(output_tags), max_seq_length, n_output_words), dtype='float32')
output_data = np.zeros(shape=(len(output_tags), max_seq_length), dtype='float32')

for i, tags in enumerate(output_tags):
    for j, tag in enumerate(tags):
        input_data[i, j, w2i_output[tag]] = 1.
        if j > 0:
            output_data[i, j-1] = w2i_output[tag]

In [8]:
learning_rate = 1e-2
target_size = image_data.shape[1]

In [9]:
train_size = 1000
valid_size = 200
batch_size = 100
n_batch = train_size // batch_size
n_valid = valid_size // batch_size

In [10]:
train_words = image_word_data[:train_size]
train_inputs = input_data[:train_size]
train_outputs = output_data[:train_size]
valid_words = image_word_data[train_size:train_size+valid_size]
valid_inputs = np.zeros(shape=(valid_size, max_seq_length, n_output_words), dtype='float32')
valid_inputs[:, 0, w2i_output['\t']] = 1.
valid_outputs = output_data[train_size:train_size+valid_size]

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [24]:
tf.reset_default_graph()

n_hidden = 256
n_input_depth = 128

enc_words = tf.placeholder(tf.float32, [None, n_image_words])
dec_inputs = tf.placeholder(tf.float32, [None, max_seq_length, n_output_words])
dec_outputs = tf.placeholder(tf.int32, [None, max_seq_length])

W = tf.Variable(tf.truncated_normal(shape=[n_image_words, max_seq_length * n_input_depth], stddev=5e-2))
b = tf.Variable(tf.constant(0.1, shape=[max_seq_length * n_input_depth]))
# enc_inputs = tf.nn.relu(tf.matmul(enc_words, W) + b)
enc_inputs = tf.matmul(enc_words, W) + b
enc_inputs = tf.reshape(enc_inputs, [-1, max_seq_length, n_input_depth])

enc_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.8)
outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_inputs, 
                                        dtype=tf.float32, scope='encoder')

dec_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.8)
outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_inputs,
                                       initial_state=enc_states,
                                       dtype=tf.float32, scope='decoder')

model = tf.layers.dense(outputs, n_output_words, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=dec_outputs))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

prediction = tf.argmax(model, 2)

In [25]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [26]:
total_epoch = 100

for epoch in tqdm(range(total_epoch)):
    total_loss = 0
#     for batch in tqdm(range(n_batch)):
    for batch in range(n_batch):
        batch_enc_inputs = train_words[batch*batch_size:(batch+1)*batch_size]
        batch_dec_inputs = train_inputs[batch*batch_size:(batch+1)*batch_size]
        batch_dec_outputs = train_outputs[batch*batch_size:(batch+1)*batch_size]
        
        _, loss = sess.run([optimizer, cost], feed_dict={
                enc_words: batch_enc_inputs,
                dec_inputs: batch_dec_inputs,
                dec_outputs: batch_dec_outputs
            })
        
        total_loss += loss
    
    total_valid_loss = 0
    for batch in range(n_valid):
        val_enc_inputs = valid_words[batch*batch_size:(batch+1)*batch_size]
        val_dec_inputs = valid_inputs[batch*batch_size:(batch+1)*batch_size]
        val_dec_outputs = valid_outputs[batch*batch_size:(batch+1)*batch_size]
        
        val_loss = sess.run(cost, feed_dict={
                enc_words: val_enc_inputs,
                dec_inputs: val_dec_inputs,
                dec_outputs: val_dec_outputs
            })
        total_valid_loss += val_loss
    
    print('Epoch: %04d Cost: %.6f Valid Cost: %.6f' %(epoch + 1, total_loss / n_batch, total_valid_loss / n_valid))

  1%|          | 1/100 [00:06<10:07,  6.13s/it]

Epoch: 0001 Cost: 4.537679 Valid Cost: 2.710578


  2%|▏         | 2/100 [00:11<09:28,  5.80s/it]

Epoch: 0002 Cost: 2.495208 Valid Cost: 2.903804


  3%|▎         | 3/100 [00:16<08:59,  5.56s/it]

...

Epoch: 0098 Cost: 0.034811 Valid Cost: 3.507264


 99%|█████████▉| 99/100 [08:17<00:05,  5.01s/it]

Epoch: 0099 Cost: 0.033423 Valid Cost: 3.511145


100%|██████████| 100/100 [08:22<00:00,  5.02s/it]

Epoch: 0100 Cost: 0.031300 Valid Cost: 3.642672


In [80]:
from konlpy.tag import Komoran

komoran = Komoran()
use_tags = ['V', 'N', 'M', 'IC', 'XR']

def translate(sentences):
    original_tags = [komoran.pos(s) for s in sentences]
    sentence_tags = []
    
    for tags in original_tags:
        sentence_tag = []
        for tag in tags:
            for use_tag in use_tags:
                if tag[1].startswith(use_tag):
                    sentence_tag.append(tag)
        sentence_tags.append(sentence_tag)
        
    enc_in = np.zeros(shape=(len(sentence_tags), n_image_words), dtype='float32')

    for i, tags in enumerate(sentence_tags):
        for tag in tags:
            enc_in[i, w2i_image[tag[0]]] += 1
            
    dec_in = np.zeros(shape=(len(sentence_tags), max_seq_length, n_output_words), dtype='float32')
    dec_in[:, 0, w2i_output['\t']] = 1.
    
    results = sess.run(prediction,
                      feed_dict={enc_words: enc_in,
                                 dec_inputs: dec_in})
    
    decoded = [[output_words[i] for i in result] for result in results]
    translated = []
    for result in decoded:
        end = result.index('\n') if '\n' in result else len(result)
        translated.append(' '.join(result[:end]))
    return translated

In [105]:
sentences = sentence_data[0:10]
    
for i, (s, p) in enumerate(zip(sentences, translate(sentences))):
    print('원본')
    print(s[1:-1])
    print(sentence_tags[i])
    print('결과')
    print(p)
    print()

원본
맛집 오늘의 포인트 이없다는거
[('맛집', 'NNG'), ('오늘', 'NNG'), ('포인트', 'NNP'), ('이', 'NP'), ('없', 'VA'), ('거', 'NNB')]
결과
역시 꽤 끝 랑 뭐 너무

원본
오늘 하늘 너무 이쁘다
[('오늘', 'NNG'), ('하늘', 'NNG'), ('너무', 'MAG'), ('이쁘', 'VA')]
결과
오늘 하늘 너무

원본
우오오오오옹 전역 일차 날씨 짱좋다아아아아
[('우', 'NNP'), ('오오', 'NNP'), ('오오', 'NNP'), ('옹', 'NNB'), ('전역', 'NNP'), ('일차', 'NNG'), ('날씨', 'NNP'), ('짱', 'MAG'), ('좋', 'VA'), ('아아', 'NNG'), ('아아', 'NNP')]
결과
우 우 복의 사람 전역 이 전역 쓰 어도 아아 아아

원본
하늘이너무이쁘다
[('하늘', 'NNG'), ('너무', 'MAG'), ('이쁘', 'VA')]
결과
하늘 이 하늘

원본
하늘이 맑고 푸르다
[('하늘', 'NNG'), ('맑', 'VA'), ('푸르', 'VA')]
결과
오랜만 하늘 맑 푸르

원본
얼마만에 이런 하늘이야 넘 좋다
[('얼마', 'NNG'), ('이런', 'MM'), ('하늘', 'NNG'), ('넘', 'NNG'), ('좋', 'VA')]
결과
얼마 쌀쌀 이야 파티 ㄴ 엄마 어

원본
오랜만에 맑은 하늘
[('오랜만', 'NNG'), ('맑', 'VA'), ('하늘', 'NNG')]
결과
오랜만 하늘 맑 고

원본
오우 오늘 무슨일 이쁘다증말 잠시김밥사러 산책 쫌만걸어도배가너무땡겨
[('오우', 'NNP'), ('오늘', 'NNG'), ('무슨', 'MM'), ('일', 'NNP'), ('이쁘', 'VA'), ('증말', 'NNG'), ('잠시', 'NNG'), ('김밥', 'NNP'), ('살', 'VV'), ('산책', 'NNP'), ('쪼', 'VV'), ('걸', 'VV'), ('어도', 'NNP'), ('배가'

In [44]:
sentences

array(['\t맛집 오늘의 포인트 이없다는거\n'], dtype=object)

'가'

In [13]:
sess.close()

In [16]:
enc_in.shape

(1, 14991)

In [21]:
sentences = sentence_data[:1]

original_tags = [komoran.pos(s) for s in sentences]
sentence_tags = []

for tags in original_tags:
    sentence_tag = []
    for tag in tags:
        for use_tag in use_tags:
            if tag[1].startswith(use_tag):
                sentence_tag.append(tag)
    sentence_tags.append(sentence_tag)

enc_in = np.zeros(shape=(len(sentence_tags), n_image_words), dtype='float32')

for i, tags in enumerate(sentence_tags):
    for tag in tags:
        enc_in[i, w2i_image[tag[0]]] += 1

dec_in = np.zeros(shape=(len(sentence_tags), max_seq_length, n_output_chars), dtype='float32')
dec_in[:, 0, c2i['\t']] = 1.

results = sess.run(model,
                  feed_dict={enc_words: enc_in,
                             dec_inputs: dec_in})

# decoded = [[c2i[i] for i in result] for result in results]
# translated = []
# for result in decoded:
#     end = result.index['\n'] if '\n' in result else len(result)
#     translated.append(''.join(result[:end]))
# translated

In [21]:
sess.run(prediction, feed_dict={enc_words: enc_in, dec_inputs: dec_in})

InternalError: 2 root error(s) found.
  (0) Internal: Dst tensor is not initialized.
	 [[{{node _arg_Placeholder_1_0_1}}]]
  (1) Internal: Dst tensor is not initialized.
	 [[{{node _arg_Placeholder_1_0_1}}]]
	 [[ArgMax_1/_149]]
0 successful operations.
0 derived errors ignored.

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tf.reset_default_graph()

Instructions for updating:
non-resource variables are not supported in the long term


In [28]:
train_image = image_data[:train_size]
train_words = image_word_data[:train_size]
valid_image = image_data[train_size:train_size+valid_size]
valid_words = image_word_data[train_size:train_size+valid_size]

In [20]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [17]:
# 이미지 (CNN)

X = tf.placeholder(tf.float32, shape=[None, target_size, target_size, 3])
Y = tf.placeholder(tf.float32, shape=[None, n_image_words])

n_conv_1 = 64
k_conv_1 = 5
W_conv_1 = tf.Variable(tf.truncated_normal(shape=[k_conv_1,k_conv_1,3,n_conv_1], stddev=5e-2))
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[n_conv_1]))
h_conv_1 = tf.nn.relu(tf.nn.conv2d(X, W_conv_1, strides=[1,1,1,1], padding='SAME') + b_conv_1)
h_pool_1 = tf.nn.max_pool(h_conv_1, ksize=[1,4,4,1], strides=[1,4,4,1], padding='SAME')

n_conv_2 = 64
k_conv_2 = 5
W_conv_2 = tf.Variable(tf.truncated_normal(shape=[k_conv_2,k_conv_2,n_conv_1,n_conv_2], stddev=5e-2))
b_conv_2 = tf.Variable(tf.constant(0.1, shape=[n_conv_2]))
h_conv_2 = tf.nn.relu(tf.nn.conv2d(h_pool_1, W_conv_2, strides=[1,1,1,1], padding='SAME') + b_conv_2)
h_pool_2 = tf.nn.max_pool(h_conv_2, ksize=[1,4,4,1], strides=[1,4,4,1], padding='SAME')

n_conv_3 = 128
k_conv_3 = 3
W_conv_3 = tf.Variable(tf.truncated_normal(shape=[k_conv_3,k_conv_3,n_conv_2,n_conv_3], stddev=5e-2))
b_conv_3 = tf.Variable(tf.constant(0.1, shape=[n_conv_3]))
h_conv_3 = tf.nn.relu(tf.nn.conv2d(h_pool_2, W_conv_3, strides=[1,1,1,1], padding='SAME') + b_conv_3)
h_pool_3 = tf.nn.max_pool(h_conv_3, ksize=[1,4,4,1], strides=[1,4,4,1], padding='SAME')

n_conv_4 = 128
k_conv_4 = 3
W_conv_4 = tf.Variable(tf.truncated_normal(shape=[k_conv_4,k_conv_4,n_conv_3,n_conv_4], stddev=5e-2))
b_conv_4 = tf.Variable(tf.constant(0.1, shape=[n_conv_4]))
h_conv_4 = tf.nn.relu(tf.nn.conv2d(h_pool_3, W_conv_4, strides=[1,1,1,1], padding='SAME') + b_conv_4)

n_conv_5 = 128
k_conv_5 = 3
W_conv_5 = tf.Variable(tf.truncated_normal(shape=[k_conv_5,k_conv_5,n_conv_4,n_conv_5], stddev=5e-2))
b_conv_5 = tf.Variable(tf.constant(0.1, shape=[n_conv_5]))
h_conv_5 = tf.nn.relu(tf.nn.conv2d(h_conv_4, W_conv_5, strides=[1,1,1,1], padding='SAME') + b_conv_5)
h_conv_5_flat = tf.reshape(h_conv_5, [-1, h_conv_5.shape[1]*h_conv_5.shape[2]*h_conv_5.shape[3]])

n_fc_1 = 256
k_conv_1 = 5
W_fc_1 = tf.Variable(tf.truncated_normal(shape=[h_conv_5_flat.shape[1], n_fc_1], stddev=5e-2))
b_fc_1 = tf.Variable(tf.constant(0.1, shape=[n_fc_1]))
h_fc_1 = tf.nn.relu(tf.matmul(h_conv_5_flat, W_fc_1) + b_fc_1)

h_fc_1_drop = tf.nn.dropout(h_fc_1, 0.8)

W_final = tf.Variable(tf.truncated_normal(shape=[n_fc_1, n_image_words], stddev=5e-2))
b_final = tf.Variable(tf.constant(0.1, shape=[n_image_words]))
model = tf.nn.relu(tf.matmul(h_fc_1_drop, W_final) + b_final)

cost = tf.reduce_mean(tf.square(model - Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [44]:
for epoch in range(total_epoch):
    total_loss = 0
    # for batch in tqdm(range(n_batch)):
    for batch in range(n_batch):
        batch_image = train_image[batch*batch_size:(batch+1)*batch_size]
        batch_words = train_words[batch*batch_size:(batch+1)*batch_size]
        
        _, loss = sess.run([optimizer, cost], feed_dict={
                X: batch_image,
                Y: batch_words
            })
        
        total_loss += loss
    
    total_valid_loss = 0
    for batch in range(n_valid):
        batch_image = valid_image[batch*batch_size:(batch+1)*batch_size]
        batch_words = valid_words[batch*batch_size:(batch+1)*batch_size]
        valid_loss = sess.run(cost, feed_dict={
                X: batch_image,
                Y: batch_words
            })
        total_valid_loss += valid_loss
    
    print('Epoch: %04d Cost: %.6f Valid Cost: %.6f' %(epoch + 1, total_loss / n_batch, valid_loss))

Epoch: 0001 Cost: 0.005442 Valid Cost: 0.005904
Epoch: 0002 Cost: 0.005441 Valid Cost: 0.005902
Epoch: 0003 Cost: 0.005439 Valid Cost: 0.005900
Epoch: 0004 Cost: 0.005437 Valid Cost: 0.005898
Epoch: 0005 Cost: 0.005435 Valid Cost: 0.005897
Epoch: 0006 Cost: 0.005434 Valid Cost: 0.005895
Epoch: 0007 Cost: 0.005432 Valid Cost: 0.005893
Epoch: 0008 Cost: 0.005430 Valid Cost: 0.005891
Epoch: 0009 Cost: 0.005428 Valid Cost: 0.005890
Epoch: 0010 Cost: 0.005426 Valid Cost: 0.005888
Epoch: 0011 Cost: 0.005425 Valid Cost: 0.005886
Epoch: 0012 Cost: 0.005423 Valid Cost: 0.005884
Epoch: 0013 Cost: 0.005421 Valid Cost: 0.005883
Epoch: 0014 Cost: 0.005419 Valid Cost: 0.005881
Epoch: 0015 Cost: 0.005418 Valid Cost: 0.005879
Epoch: 0016 Cost: 0.005416 Valid Cost: 0.005877
Epoch: 0017 Cost: 0.005414 Valid Cost: 0.005876
Epoch: 0018 Cost: 0.005413 Valid Cost: 0.005874
Epoch: 0019 Cost: 0.005411 Valid Cost: 0.005872
Epoch: 0020 Cost: 0.005409 Valid Cost: 0.005871
Epoch: 0021 Cost: 0.005407 Valid Cost: 0

KeyboardInterrupt: 

In [51]:
def test(images, word_data):
    pred_words = sess.run(model, feed_dict={
            X: images
        })
    
    y = [{image_words[i]: n for i, n in enumerate(words) if n > 0} for words in word_data]
    p = [{image_words[i]: '%.3f' %n for i, n in enumerate(words) if n > 0} for words in pred_words]
    
    for _y, _p in zip(y, p):
        print(_y)
        print(_p)
        print()

In [53]:
test(train_image[:1], image_word_data[:1])

{'가': 1.0, '가사': 1.0, '같': 1.0, '거리': 1.0, '격상': 1.0, '광주': 1.0, '구청': 1.0, '그대': 2.0, '기다리': 1.0, '기대': 1.0, '꼭': 1.0, '나무': 2.0, '내': 2.0, '단계': 1.0, '대교': 1.0, '되': 1.0, '되찾': 1.0, '두기': 1.0, '때': 1.0, '마스크': 1.0, '및': 1.0, '방역': 2.0, '보': 1.0, '사진': 1.0, '사회': 1.0, '수': 1.0, '수칙': 2.0, '쉬': 1.0, '양요섭': 1.0, '언제나': 1.0, '여기': 1.0, '여행': 1.0, '우리': 1.0, '위하': 1.0, '일상': 1.0, '있': 2.0, '잠시': 1.0, '전': 1.0, '주': 2.0, '준수': 2.0, '중': 1.0, '지키': 1.0, '착용': 1.0, '천주교': 1.0, '촬영': 1.0, '힘들': 1.0}
{'ㄹ': '0.027', '가': '0.035', '가가': '0.044', '가게': '0.034', '가게웨이팅해서': '0.015', '가격': '0.034', '가계': '0.005', '가계부': '0.034', '가고싶옹': '0.002', '가고파': '0.033', '가공': '0.028', '가구': '0.029', '가까이': '0.034', '가까히': '0.008', '가깝': '0.034', '가꾸': '0.017', '가끔': '0.033', '가끔가다': '0.030', '가나': '0.027', '가나슈': '0.033', '가난': '0.015', '가념일': '0.030', '가느다랗': '0.018', '가늘': '0.034', '가늠': '0.034', '가능': '0.045', '가니': '0.034', '가니쉬': '0.034', '가도': '0.034', '가돈': '0.033', '가동': '0.033', '가두': '0.008', '가드':